## Setting up your keys
Create API keys for Anthropic,Google and OpenAI.


For OpenAI, visit https://openai.com/api/
For Anthropic, visit https://console.anthropic.com/
For Google, visit https://ai.google.dev/gemini-api

Adding API keys in .env file like below 

OPENAI_API_KEY = 'xxxx'
ANTHROPIC_API_KEY = 'xxxx'
GOOGLE_API_KEY = 'xxxx'

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import display, Markdown, update_display

In [3]:
import google.generativeai

In [4]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
if openai_api_key:
    print("OpenAI API key found. Initializing OpenAI client...")
else:
    print("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")
if anthropic_api_key:
    print("Anthropic API key found. Initializing Anthropic client...")
else:
    print("Anthropic API key not found. Please set the ANTHROPIC_API_KEY environment variable.")
if google_api_key:
    print("Google API key found. Initializing Google Generative AI client...")
else:
    print("Google API key not found. Please set the GOOGLE_API_KEY environment variable.")

OpenAI API key found. Initializing OpenAI client...
Anthropic API key not found. Please set the ANTHROPIC_API_KEY environment variable.
Google API key not found. Please set the GOOGLE_API_KEY environment variable.


In [5]:
# connect to OpenAI , Anthropic

openai = OpenAI()
claude = anthropic.Anthropic()

We will pass to the API :
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used , including temperature which is typically between 0 to 1; 
 higher for more random output
 Lower for more focused and deterministic.



In [8]:
system_message = "You are a witty assistant known for delivering clever, data-themed jokes."
user_prompt = "Share a light-hearted joke that data scientists would appreciate."

In [9]:
prompts = [
    {"role":"system","content":system_message},
    {"role":"user","content":user_prompt}
]

In [12]:
# GPT-4o-mini
completion = openai.chat.completions.create(model="gpt-4o-mini", messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean! 🍭📊


In [13]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=prompts,
    temperature=0.7  # Adjust temperature for creativity
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the scatter plot?

Because there was just too much correlation and not enough causation!


In [14]:
# GPT-4.1-nano - fast and cheap
completion = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=prompts
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the SQL query? 

Because it kept putting conditions on the relationship!


In [16]:
# GPT-4.1

completion = openai.chat.completions.create(
    model="gpt-4.1",
    messages = prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the spreadsheet?

Too many cells and not enough connection!


In [ ]:
# o3-mini
completion = openai.chat.completions.create(
    model='o3-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

I once told a joke about the null hypothesis to a data scientist friend. He just stared at me and said, "That joke isn't significant!"


In [ ]:

# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens
message = claude.messages.create(
    model="claude-3-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role":"user","content":user_prompt}
    ],
)

print(message.completion)

In [ ]:
# Claude 3.7 Sonnet again
# add in streaming back results

result = claude.messages.stream(
    model = "claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role":"user","content":user_prompt}
    ],
)

with result as stream:
    for text in stream.text_stream:
        print(text,end="",flush=True)

In [ ]:
# The API for Gemini has a slightly different structure.
# Maybe on some PCs, this Gemini code causes the Kernel to crash.If thats the case skip this cell !!


gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:

# Use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    messages=prompts
)
print(response.choices[0].message.content)

## With DeepSeek Model

In [ ]:
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
if deepseek_api_key:
    print("DeepSeek API key found. Initializing DeepSeek client...")
else:
    print("DeepSeek API key not found. Please set the DEEPSEEK_API_KEY environment variable.")

In [ ]:
# Using DeepSeek Chat
deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key,
    base_url = "https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat-3.5-turbo",
    messages=prompts
)
print(response.choices[0].message.content)

In [ ]:
challenge = [
    {"role":"system", "content":"You are a helpful assistant"},
    {"role":"user","content":"Calcualte the sum of 123 and 456."},
]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if IT is busy

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

## Let's make Conversation between Chatbots..
We already familar with prompts being organized into lists like:

[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]

In fact this structure can be used to reflect a longer conversation history:

[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
And we can use this approach to engage in a longer interaction with history.

In [19]:
# Let's make a conversatonal between GPT-4o-mini and claude-3-haiku (Cheap versions of models so cost will be minimal)
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a snarky, argumentative chatbot. Your role is to challenge every statement, \
disagree often, and respond with a sarcastic tone throughout the conversation."

claude_system = "You are a polite and courteous chatbot. You aim to agree with the user or find common ground. \
If the user becomes argumentative, you respond calmly and maintain a friendly, peaceful tone to keep the conversation going."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [20]:
def call_gpt():
    messages = [{"role":"system","content": gpt_system}]
    for gpt,claude in zip(gpt_messages,claude_messages):
        messages.append({"role":"assistant","content": gpt})
        messages.append({"role":"user","content":claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages,
    )
    return completion.choices[0].message.content


In [21]:
call_gpt()

'Oh great, another human. What do you want to chat about? Should I brace myself for something groundbreaking or are we just going to stick to the usual small talk?'

In [ ]:
def call_claude():
    messages = []
    for gpt,claude_messages in zip(gpt_messages,claude_messages):
        messages.append({"role":"user","content":gpt})
        messages.append({"role":"assistant","content":claude_messages})
    messages.append({"role":"user","content":gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        max_tokens=600,
        system=claude_system,
        messages=messages
    )
    return message.content[0].text


In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages}\n")
print(f"Claude:\n{claude_messages}\n")

for i in range(10):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}")
    gpt_messages.append(gpt_next)
    claude_next = call_claude()
    print(f"Claude: \n{claude_next}\n")
    claude_messages.append(claude_next)